In [ ]:
import json
from typing import List, Literal, Tuple

import pymupdf

from parser.elements.rectangle import Rectangle

PyMuPDFRect = Tuple[float, float, float, float, str, float, float, float]

InputAreaKind = Literal[
    "underscores",
    "code_underscores",
    "textarea",
    "code_textarea",
    "entire_function_area",
]


def rect_to_bbox(rect: PyMuPDFRect) -> Tuple[float, float, float, float]:
    return (rect[0], rect[1], rect[2], rect[3])


def mark_rect(page: pymupdf.Page, rect: PyMuPDFRect | Rectangle, kind: InputAreaKind):
    """Underline each word that contains 'text'."""
    if isinstance(rect, Rectangle):
        bbox = rect.as_tuple()
    else:
        bbox = rect_to_bbox(rect)

    annot = page.add_rect_annot(bbox)  # underline
    blue = (0, 0, 1)
    if kind == "underscores":
        fill = (153 / 255, 240 / 255, 234 / 255)
    elif kind == "textarea":
        fill = (234 / 255, 240 / 255, 153 / 255)
    elif kind == "code_textarea":
        fill = (234 / 255, 153 / 255, 240 / 255)
    elif kind == "code_underscores":
        fill = (153 / 255, 240 / 255, 234 / 255)
    elif kind == "entire_function_area":
        fill = (234 / 255, 153 / 255, 240 / 255)
    else:
        raise ValueError(f"Invalid kind: {kind}")

    annot.set_border(width=1, dashes=[1, 2])
    annot.set_colors(stroke=blue, fill=fill)
    annot.update(opacity=0.5)


def serialize_blocks_to_json(
    blocks: List[PyMuPDFRect],
) -> str:
    blocks_list = [
        {
            "x0": block[0],
            "y0": block[1],
            "x1": block[2],
            "y1": block[3],
            "text": block[4],
            "width": block[5],
            "height": block[6],
        }
        for block in blocks
    ]
    return json.dumps(blocks_list, indent=4)

In [ ]:
import os
from typing import List, Tuple


def load_exam_and_extract_text_words(filepath: str):
    doc = pymupdf.open(filepath)

    for i, page in enumerate(doc):  # scan through the pages
        blocks: List[Tuple[float, float, float, float, str, float, float]] = (
            page.get_text("blocks")
        )

        for block in blocks:
            mark_rect(page, block)  # mark the page's words

        json_data = serialize_blocks_to_json(blocks)
        with open(f"page-{i}-blocks.json", "w") as json_file:
            json_file.write(json_data)

    doc.save("marked-" + os.path.basename(filepath))


load_exam_and_extract_text_words("../fe_files/exams/FE-Jan24.pdf")

In [ ]:
import json  # noqa: F811
import os  # noqa: F811
from typing import Literal

import pymupdf  # noqa: F811
import regex
from pydantic import BaseModel

from parser.elements.rectangle import (
    Rectangle,
    build_connectivity_graph,
    find_connected_components,
)

MARKED_RECTS: List[Rectangle] = []


class SeparatedRects(BaseModel):
    underscores: PyMuPDFRect | None
    remaining: PyMuPDFRect | None


def separate_underscores_from_text(
    page: pymupdf.Page, rect: Rectangle, text: str
) -> SeparatedRects:
    underscores = None
    remaining = None

    # Find the longest continuous sequence of underscores
    longest_underscore_seq = ""
    current_seq = ""

    for char in text:
        if char == "_":
            current_seq += char
        else:
            if len(current_seq) > len(longest_underscore_seq):
                longest_underscore_seq = current_seq
            current_seq = ""

    # Check the last sequence
    if len(current_seq) > len(longest_underscore_seq):
        longest_underscore_seq = current_seq

    if len(longest_underscore_seq) >= 4:
        # Use page.search_for to find the remaining text
        non_underscores = text.replace(longest_underscore_seq, "")
        remaining_bbox = page.search_for(
            text.replace(longest_underscore_seq, ""), clip=rect.as_tuple()
        )
        if remaining_bbox:
            remaining = (
                remaining_bbox[0][0],
                remaining_bbox[0][1],
                remaining_bbox[0][2],
                remaining_bbox[0][3],
                non_underscores,
                0,
                0,
                0,
            )

        underscores_bbox = page.search_for(longest_underscore_seq, clip=rect.as_tuple())
        if underscores_bbox:
            underscores = (
                underscores_bbox[0][0],
                underscores_bbox[0][1],
                underscores_bbox[0][2],
                underscores_bbox[0][3],
                longest_underscore_seq,
                0,
                0,
                0,
            )
    else:
        remaining = None

    return SeparatedRects(underscores=underscores, remaining=remaining)


def merge_rectangles(rectangles: List[Rectangle]) -> Rectangle:
    MAX_DISTANCE = (
        2000000  # Maximum allowed distance to consider rectangles as neighbors
    )

    # Assign an index to each rectangle for easy reference
    rect_indices = {i: rect for i, rect in enumerate(rectangles)}

    # Step 1: Build the connectivity graph using the function
    graph = build_connectivity_graph(rect_indices, MAX_DISTANCE)

    # Step 2: Find connected components using the function
    components = find_connected_components(graph, rect_indices)

    # Step 3: Merge rectangles in each connected component
    merged_rectangles: List[Rectangle] = []

    for component in components:
        merged_rect = rect_indices[component[0]]

        for idx in component[1:]:
            merged_rect = merged_rect.merge_with(rect_indices[idx])

        merged_rectangles.append(merged_rect)

    # Step 4: Select the largest rectangle
    largest_rectangle: Rectangle = max(merged_rectangles, key=lambda r: r.area())

    return largest_rectangle


class PageBlocks(BaseModel):
    page_number: int
    blocks: List[PyMuPDFRect]
    merged: Rectangle
    merged_text: str

    @classmethod
    def from_page_bboxes(
        cls, page_bboxes: List[PyMuPDFRect], pymypdfPage: pymupdf.Page
    ):
        rectangles: List[Rectangle] = [
            Rectangle.from_points(*bbox[0:4]) for bbox in page_bboxes
        ]

        largest_rectangle: Rectangle = merge_rectangles(rectangles)

        return cls(
            blocks=page_bboxes,
            merged=largest_rectangle,
            merged_text=pymypdfPage.get_text(clip=largest_rectangle.as_tuple()),
            page_number=pymypdfPage.number,
        )


class CodeInputSubArea(BaseModel):
    kind: Literal["fill_in_the_blank", "free_response"]
    code_textarea: Rectangle | None
    text: str


class CodeArea(BaseModel):
    rect: Rectangle
    sub_areas: List[CodeInputSubArea]
    text: str


def filter_empty_lines(text: str) -> str:
    return text.replace(" ", "").replace("\n", "")


def detect_empty_branches(code: str) -> List[str]:
    print(f"code={code}")
    # Regular expression to match empty code branches, e.g., for loops, if conditions, etc.
    # pattern = re.compile(r"\{\s*\}")

    pattern = regex.compile(r"^\{\s*(?:\/\/.*\s*)*\}$")

    matches = pattern.findall(code)
    print(f"matches={matches}")
    for match in matches:
        print(f"match={match}")

    return matches


def extract_code_inputs(page: pymupdf.Page, page_blocks: PageBlocks) -> List[CodeArea]:
    # Find all matches

    pattern = regex.compile(
        r"""
   ^\s*                                   # Ensure the function definition starts at a line (after optional indentation)
    (?P<return_type>
        [\w\*\&\[\]]+                      # The first part of the return type (e.g., int, void, etc.)
        (?:                                 # Non-capturing group for multiple parts of the return type
            (?:\s+|/[*].*?[*]/|//.*?$)+     # Allow whitespace or comments between return type parts
            [\w\*\&\[\]]+                  # Additional part of the return type (e.g., static, inline)
        )*
    )
    \s+                                    # Some whitespace after the return type before the function name
    (?P<function_name>\w+)                 # The function name
    \s*                                    # Optional whitespace before the arguments
    \(
        (?P<arguments>
            (?:[^()]*|\((?:[^()]|\([^()]*\))*\))*
        )
    \)\s*
    (?P<body>
        \{
            (?:
                [^{}]+                     # Non-brace characters
                |
                (?&body)                   # Recursively match nested braces
            )*
        \}
    )
    """,
        regex.MULTILINE | regex.DOTALL | regex.VERBOSE,
    )

    # matches = FUNCTION_EXTRACTION_PATTERN.finditer(page_blocks.merged_text)
    matches = pattern.finditer(page_blocks.merged_text)

    code_areas: List[CodeArea] = []

    print(f"page_number={page_blocks.page_number}")
    for match in matches:
        code_sub_areas: List[CodeInputSubArea] = []

        raw = match.group(0)
        print(f"raw={raw}")
        print(f"match={match}")
        function_name = match.group("function_name")
        # arguments = match.group("arguments")
        body = match.group("body")

        empty_branches: List[str] = detect_empty_branches(body)
        if len(empty_branches) > 0:
            print(f"empty_branches={empty_branches}")
            for branch in empty_branches:
                code_bboxes = page.search_for(
                    branch, clip=page_blocks.merged.as_tuple()
                )
                if code_bboxes is None or len(code_bboxes) == 0:
                    raise ValueError(
                        f"No code bbox found for body='{body}', branch={branch}"
                    )

                code_textarea_rect = merge_rectangles(
                    [Rectangle.from_points(*x[0:4]) for x in code_bboxes]
                )

                # code_textarea_rect = Rectangle.from_points(*code_bboxes[0][0:4])
                code_sub_areas.append(
                    CodeInputSubArea(
                        kind="free_response",
                        code_textarea=code_textarea_rect,
                        text=body,
                    )
                )
                if function_name is not None and function_name == "triangularSum":
                    print(f"Found triangularSum: {body}")

        if len(code_sub_areas) > 0:
            merged_code_area = merge_rectangles(
                [
                    Rectangle.from_points(*x[0:4])
                    for x in page.search_for(raw, clip=page_blocks.merged.as_tuple())
                ]
            )
            # merged_code_area = merge_rectangles(
            #    [x.code_textarea for x in code_sub_areas if x.code_textarea is not None]
            # )

            code_area = CodeArea(
                rect=merged_code_area, text=raw, sub_areas=code_sub_areas
            )
            code_areas.append(code_area)

    return code_areas


def is_code_page(
    page: pymupdf.Page, page_blocks: PageBlocks
) -> Tuple[bool, List[CodeArea]]:
    code: List[CodeArea] = extract_code_inputs(page, page_blocks)
    if len(code) > 0:
        return True, code
    return False, None


def load_exam_and_extract_text_words(filepath: str):
    doc = pymupdf.open(filepath)

    for i, page in enumerate(doc):  # scan through the pages
        blocks: List[PyMuPDFRect] = page.get_text("words")

        page_blocks = PageBlocks.from_page_bboxes(blocks, page)

        underscore_marking_queue: List[Rectangle] = []

        for block in page_blocks.blocks:
            rectangle = Rectangle.from_points(*block[0:4])
            rect_text = block[4]
            separated = separate_underscores_from_text(page, rectangle, rect_text)
            if separated.underscores is not None:
                underscore_marking_queue.append(separated.underscores)
                # mark_rect(page, separated.underscores, "underscores")

        is_code, code_areas = is_code_page(page, page_blocks)
        print(f"is_code={is_code}")

        if is_code:
            assert code_areas is not None
            for code_area in code_areas:
                assert len(code_area.sub_areas) > 0
                print(f"underscore_marking_queue={underscore_marking_queue}")

                mark_rect(page, code_area.rect, "entire_function_area")

                for underscore in underscore_marking_queue:
                    underscore_rect = Rectangle.from_points(*rect_to_bbox(underscore))
                    if underscore_rect.is_within(code_area.rect):
                        # remove the underscore from the queue
                        mark_rect(page, underscore_rect, "code_underscores")
                        print(f"Removed underscore from queue: {underscore}")
                        underscore_marking_queue.remove(underscore)

                for area in code_area.sub_areas:
                    mark_rect(page, area.code_textarea, "code_textarea")

            with open(f"page-{i}-code_areas.json", "w") as code_file:
                json.dump(
                    [code_area.model_dump() for code_area in code_areas],
                    code_file,
                    indent=4,
                )

        for underscore in underscore_marking_queue:
            mark_rect(
                page,
                Rectangle.from_points(*rect_to_bbox(underscore)),
                "code_underscores",
            )

        with open(f"page-{i}-blocks.json", "w") as json_file:
            json_file.write(page_blocks.model_dump_json(indent=4))

    doc.save("marked-words-" + os.path.basename(filepath))


load_exam_and_extract_text_words("../fe_files/exams/FE-May24.pdf")